## -- 1. 样本介绍 & 数据读取 --
**Ref: https://github.com/NLP-LOVE/ML-NLP** <br>
/Users/samxie/Research/Match_AI/Discuss/Product_Creator_In_20250407_220618.csv <br>
/Users/samxie/Research/Match_AI/Discuss/Creator_List.csv <br>
**聚类：商品类别 verse 达人类，寻找两类关系** <br>

### 达人维度数据:
Affiliate GMV, <br> 
Affiliate LIVE GMV, <br>
Affiliate shoppable video GMV, <br>
Affiliate product card, <br> 
GMV	Affiliate products sold, <br>
Items sold,	<br>
Est. commission, <br>
Avg. order value, <br>
Affiliate product showcase,	<br>
Affiliate orders, <br>
CTR, <br>
Product impressions, <br>
Avg. affiliate customers, <br>
Affiliate LIVEs, <br>
Affiliate shoppable videos,	<br>
Target collaboration GMV, <br>
Target collaboration est. commission, <br>
Open collaboration GMV,	<br>
Open collaboration est. commission,	<br>
Affiliate refunded GMV,	<br>
Affiliate items refunded, <br>
Affiliate followers, <br>
ROI <br>
### 商品维度数据:
视频标题, <br>
视频时长, <br>
达人粉丝数, <br>
国家/地区, <br>
发布时间, <br>
播放量, <br>
点赞数, <br>
互动率, <br>
该商品销量, <br>
该商品销售额 <br>

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy import sparse
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
creator_df = pd.read_csv('/Users/samxie/Research/Match_AI/Discuss/Creator_List.csv')
creator_df.head()

,Creator username,Affiliate GMV,Affiliate LIVE GMV,Affiliate shoppable video GMV,Affiliate product card GMV,Affiliate products sold,Items sold,Est. commission,Avg. order value,Affiliate product showcase,...,Affiliate LIVEs,Affiliate shoppable videos,Target collaboration GMV,Target collaboration est. commission,Open collaboration GMV,Open collaboration est. commission,Affiliate refunded GMV,Affiliate items refunded,Affiliate followers,ROI
0,dinastee,2.130662e+08,0.00,2.129886e+08,77566.0,8,2557,19214046.00,85466.00,0,...,0,5,0.00,0.00,2.125906e+08,9906956.00,29632232.00,340,30174.0,10.089084
1,darleneserrano2,7.628796e+05,1239.06,7.598715e+05,1769.0,5,2543,27050.05,307.86,0,...,4,11,0.00,0.00,7.628796e+05,12224.86,80733.89,269,8264.0,27.202519
2,jomaryeedump_,6.342531e+05,0.00,6.325831e+05,1670.0,6,2159,49543.80,303.33,0,...,0,0,0.00,0.00,6.342531e+05,49543.80,78457.22,268,2561482.0,11.801867
3,anjbbln,3.675751e+05,0.00,3.627377e+05,4837.4,10,1642,28144.23,256.87,0,...,0,14,0.00,0.00,3.675751e+05,28144.23,60127.53,168,25981.0,12.060407
4,iamadrian07,4.266763e+05,406364.46,1.995083e+04,361.0,28,1541,41039.45,331.01,0,...,31,0,187820.29,22732.99,2.388560e+05,18275.00,103645.27,255,375665.0,9.396735


In [3]:
product_df = pd.read_csv('/Users/samxie/Research/Match_AI/Discuss/Product_Creator_In_20250407_220618.csv')
product_df.head()

,Creator username,Affiliate GMV,Affiliate LIVE GMV,Affiliate shoppable video GMV,Affiliate product card GMV,Affiliate products sold,Items sold,Est. commission,Avg. order value,Affiliate product showcase,...,FastMoss达人详情页链接,TikTok官网达人主页链接,发布时间,播放量,点赞数,互动率,FastMoss视频详情页链接,该商品销量,该商品销售额,TikTok官网视频主页
0,jomaryeedump_,634253.12,0.00,632583.12,1670.0,6,2159,49543.80,303.33,0,...,www.fastmoss.com/zh/influencer/detail/67394995...,https://www.tiktok.com/@,2024/11/9,1500000,39700,0.028,www.fastmoss.com/zh/media-source/video/7435206...,312,219408.96,https://www.tiktok.com/@/video/743520644713220...
1,iamadrian07,426676.29,406364.46,19950.83,361.0,28,1541,41039.45,331.01,0,...,www.fastmoss.com/zh/influencer/detail/69903791...,https://www.tiktok.com/@,2024/9/10,512094,6458,0.014,www.fastmoss.com/zh/media-source/video/7412978...,16,11705.87,https://www.tiktok.com/@/video/741297832171523...
2,goojodoq.philippines,382191.08,377616.02,4575.06,0.0,20,843,33769.81,557.94,0,...,www.fastmoss.com/zh/influencer/detail/72515038...,https://www.tiktok.com/@,2025/1/24,19300,58,0.004,www.fastmoss.com/zh/media-source/video/7463426...,22,14364.31,https://www.tiktok.com/@/video/746342698209771...
3,skincarechuchay,51883.46,0.00,51883.46,0.0,4,272,3964.31,210.91,0,...,www.fastmoss.com/zh/influencer/detail/18592546,https://www.tiktok.com/@,2025/3/7,1276,8,0.013,www.fastmoss.com/zh/media-source/video/7479035...,1,698.00,https://www.tiktok.com/@/video/747903557058963...
4,y0collections_,77445.22,77325.22,120.00,0.0,4,187,7329.00,490.16,0,...,www.fastmoss.com/zh/influencer/detail/72963117...,https://www.tiktok.com/@,2025/3/14,1024,4,0.007,www.fastmoss.com/zh/media-source/video/7481330...,2,1394.72,https://www.tiktok.com/@/video/748133002857213...


## -- 2. 特征工程 --

### -- 2.1 商品侧 --

In [5]:
## 2.1.1 BERT 编码 “视频标题”
device = torch.device('cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertmodel = BertModel.from_pretrained('bert-base-uncased').to(device)
bertmodel.eval()

def bert_encode(texts, batch_size=16, max_length=128, random_state=None):
    """
    使用 BERT 对文本列表进行编码，支持可选随机数种子以保证可重复性。

    Para：
    -----------
    - texts: List[str]，要编码的文本列表。
    - batch_size: int，批大小。
    - max_length: int，最大序列长度。
    - random_state: int or None，如果不为 None，则设置 numpy 和 torch 的随机种子。

    Reture：
    -----------
    - np.ndarray, shape=(len(texts), hidden_size)，每条文本的平均池化向量。
    """
    # 可选：设置随机种子，保证可重复性
    if random_state is not None:
        np.random.seed(random_state)
        torch.manual_seed(random_state)
        torch.cuda.manual_seed_all(random_state)

    vectors = []
    for i in tqdm(range(0, len(texts), batch_size), desc="BERT Encoding"):
        batch = texts[i : i + batch_size]
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            return_tensors='pt',
            max_length=max_length
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bertmodel(**inputs)
        # 对 last_hidden_state 做平均池化
        emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        vectors.append(emb)

    return np.vstack(vectors)

titles = product_df['视频标题'].fillna('').astype(str).tolist()
title_vecs = bert_encode(titles, random_state=202504)
X_title_bert = sparse.csr_matrix(title_vecs)

BERT Encoding: 100%|████████████████████████████| 99/99 [00:24<00:00,  3.98it/s]


In [6]:
## 2.1.2 数值特征：标准化
num_prod_cols = [
    '视频时长', '达人粉丝数',
    '播放量', '点赞数', '互动率',
    '该商品销量', '该商品销售额'
]
scaler_prod = StandardScaler()
X_num_prod = scaler_prod.fit_transform(product_df[num_prod_cols].fillna(0))

In [7]:
# 2.1.3 时间特征 - 发布日期拆成月和周几
product_df['发布时间'] = pd.to_datetime(product_df['发布时间'], errors='coerce')
product_df['pub_month']   = product_df['发布时间'].dt.month.fillna(0).astype(int)
product_df['pub_weekday'] = product_df['发布时间'].dt.weekday.fillna(0).astype(int)
X_time = product_df[['pub_month', 'pub_weekday']].values

In [8]:
# 2.1.4 国家/地区 one-hot 编码
ohe_region = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_region = ohe_region.fit_transform(product_df[['国家/地区']].fillna(''))

In [9]:
# 2.1.5 合并商品特征矩阵
X_product = sparse.hstack([
    X_title_bert,
    sparse.csr_matrix(X_num_prod),
    sparse.csr_matrix(X_time),
    sparse.csr_matrix(X_region)
])
print("商品特征矩阵 X_product 形状：", X_product.shape)

商品特征矩阵 X_product 形状： (1581, 779)


### -- 2.2 达人侧特征工程 --

In [10]:
# 2.2.1 数值特征：直接标准化
num_cre_cols = [
    'Affiliate GMV', 'Affiliate LIVE GMV',
    'Affiliate shoppable video GMV', 'Affiliate product card GMV',
    'Affiliate products sold', 'Items sold', 'Est. commission',
    'Avg. order value', 'Affiliate product showcase', 'Affiliate orders',
    'CTR', 'Product impressions', 'Avg. affiliate customers',
    'Affiliate LIVEs', 'Affiliate shoppable videos',
    'Target collaboration GMV', 'Target collaboration est. commission',
    'Open collaboration GMV', 'Open collaboration est. commission',
    'Affiliate refunded GMV', 'Affiliate items refunded',
    'Affiliate followers', 'ROI'
]
scaler_cre = StandardScaler() # 标准化 Scaler, 或许可以根据数据再微调？比如取对数？

# 达人有效样本，来自于product数据集中已经和商品匹配上的达人维度信息
X_creator_eff = sparse.csr_matrix(
    scaler_cre.fit_transform(product_df[num_cre_cols].fillna(0))
)
print("有效样本达人特征矩阵 X_creator_eff 形状：", X_creator_eff.shape)

# 达人总样本，来自于Creator数据集，共有25112位达人
X_creator = sparse.csr_matrix(
    scaler_cre.fit_transform(creator_df[num_cre_cols].fillna(0))
)
print("全样本达人特征矩阵 X_creator 形状：", X_creator.shape)

有效样本达人特征矩阵 X_creator_eff 形状： (1581, 23)
全样本达人特征矩阵 X_creator 形状： (252112, 23)


## -- 3. 聚类 --

In [11]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# 假设前面已完成特征工程，得到：
#   X_product            # (n_products × D_p) 稀疏矩阵
#   product_df           # 包含 1500+ 条商品记录的 DataFrame
#   X_creator_full       # (n_all_creators × D_c) 稀疏矩阵
#   creator_df           # 包含 250k 条达人记录的 DataFrame

###  -- 3.1 商品聚类 --

In [12]:
# 3.1.1 商品聚类 - 使用MiniBatchKMeans聚类，seed选202504
K_product = 3  # 商品维度聚类选多少合适？

product_km = MiniBatchKMeans(
    n_clusters=K_product,
    random_state=202504,   # 固定随机种子为202504
    batch_size=256,
    n_init='auto'
)
product_labels = product_km.fit_predict(X_product)
product_df['ProductClusterID'] = product_labels

In [13]:
# 3.1.2 聚类质量评估（随机抽样）
def evaluate_clustering(X, labels, sample_n=500, replace=False, random_state=None):
    """
    对聚类结果进行质量评估：Silhouette 分数 & Calinski–Harabasz 分数。

    Para：
    -----------
    X : array-like or sparse matrix, shape (n_samples, n_features)
        输入的特征矩阵。
    labels : array-like, shape (n_samples,)
        对应的聚类标签。
    sample_n : int, optional (default=500)
        抽样数量，实际抽样数为 min(sample_n, n_samples)。
    replace : bool, optional (default=False)
        是否允许有放回抽样。
    random_state : int or None, optional
        随机种子，用于可重复抽样。

    Return：
    -----------
    sil : float
        Silhouette 分数。
    ch : float
        Calinski–Harabasz 分数。
    """
    n_samples = X.shape[0]
    n_draw = min(sample_n, n_samples)
    rng = np.random.default_rng(random_state)
    idx = rng.choice(n_samples, size=n_draw, replace=replace)
    
    # 如果是稀疏矩阵，则转为密集
    if hasattr(X, "toarray"):
        X_sub = X[idx].toarray()
    else:
        X_sub = X[idx]
    
    labels_sub = np.array(labels)[idx]
    
    sil = silhouette_score(X_sub, labels_sub)
    ch  = calinski_harabasz_score(X_sub, labels_sub)
    print(f"聚类评估 -> Silhouette={sil:.3f}, Calinski–Harabasz={ch:.3f}")
    return sil, ch

# 抽样评估聚类效果
evaluate_clustering(
    X_product,
    product_labels,
    sample_n=500,
    replace=False
    # , random_state=202504
)

聚类评估 -> Silhouette=0.101, Calinski–Harabasz=41.220


(0.1013358158001691, 41.22047044454243)

In [14]:
# 3.1.3 查看每个簇大小
print("商品簇分布：")
print(product_df['ProductClusterID'].value_counts().sort_index())

商品簇分布：
0    699
1    612
2    270
Name: ProductClusterID, dtype: int64


###  -- 3.2 达人聚类 --

In [15]:
# 3.2.1 达人聚类
K_creator = 4  # 根据聚类评估指标调整
creator_km = MiniBatchKMeans(
    n_clusters=K_creator,
    random_state=202505,   # 固定随机种子为 202505
    batch_size=1024,
    n_init='auto'          # 明确初始化次数，避免警告
)

# 从product得到有效样本中对达人维度聚类
creator_eff_labels = creator_km.fit_predict(X_creator_eff)
creator_eff_df = product_df[['Creator username'] + num_cre_cols]
creator_eff_df['CreatorClusterID'] = creator_eff_labels

/var/folders/x9/p1d98gj518326zdc2s5gzn8w0000gn/T/ipykernel_9813/192274762.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  creator_eff_df['CreatorClusterID'] = creator_eff_labels


In [16]:
# 3.2.2 聚类质量评估（随机抽样）
evaluate_clustering(
    X_creator_eff,
    creator_eff_labels,
    sample_n=500,
    replace=False
    # , random_state=202504
)

聚类评估 -> Silhouette=0.634, Calinski–Harabasz=163.087


(0.6338979507844058, 163.08702211083445)

In [17]:
# 3.2.3 将聚类模型应用到全量达人池
full_labels = creator_km.predict(X_creator)
creator_df['CreatorClusterID'] = full_labels

In [18]:
# 3.2.4 查看分布
print("有效达人簇分布：")
print(creator_eff_df['CreatorClusterID'].value_counts().sort_index())
print("\n全量达人簇分布：")
print(creator_df['CreatorClusterID'].value_counts().sort_index())

有效达人簇分布：
0    1238
1      76
2     230
3      37
Name: CreatorClusterID, dtype: int64

全量达人簇分布：
0    235066
1      9610
2      7072
3       364
Name: CreatorClusterID, dtype: int64


## -- 4. 簇间关系建模 --

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 通过刚刚的代码，环境中已有：——
# product_df      # 有效样本 DataFrame，含 'ProductClusterID' 列
# creator_eff_df  # 有效达人 DataFrame，含 'CreatorClusterID' 列
# creator_df      # 全量达人 DataFrame，含 'CreatorClusterID' 列
# X_product       # 稀疏矩阵，对应 product_df 行，商品特征
# num_cre_cols    # 列表，product_df 中的达人数值特征列

In [20]:
# 4.1 将有效达人簇合并到 product_df
product_df['CreatorClusterID'] = creator_eff_df['CreatorClusterID'].values
product_eff = product_df
print(product_eff[['Creator username', 'CreatorClusterID']].head())

       Creator username  CreatorClusterID
0         jomaryeedump_                 3
1           iamadrian07                 3
2  goojodoq.philippines                 3
3       skincarechuchay                 3
4        y0collections_                 3


In [21]:
# 4.2 计算簇-簇聚合统计：选取的这四个维度是否合理？
agg = product_eff.groupby(['ProductClusterID', 'CreatorClusterID']).agg(
    avg_ROI            = ('ROI',            'mean'),
    avg_GMV            = ('Affiliate GMV',  'mean'),
    total_items_sold   = ('Items sold',     'sum'),
    total_impressions  = ('Product impressions', 'sum')
).reset_index()
agg['conversion_rate'] = agg['total_items_sold'] / agg['total_impressions']

In [22]:
# 4.3 构建簇交互矩阵 M_ROI
M_ROI = agg.pivot(
    index='ProductClusterID',
    columns='CreatorClusterID',
    values='avg_ROI'
).fillna(0)
print("簇交互矩阵 M_ROI 形状：", M_ROI.shape)


簇交互矩阵 M_ROI 形状： (3, 4)


In [23]:
# 4.4 训练样本
enc_p = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
enc_c = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
P_ohe = enc_p.fit_transform(product_eff[['ProductClusterID']])
C_ohe = enc_c.fit_transform(product_eff[['CreatorClusterID']])

X_prod_dense = X_product.toarray()
X_cre_feat   = product_eff[num_cre_cols].fillna(0).values
X_all = np.hstack([P_ohe, C_ohe, X_prod_dense, X_cre_feat])
y     = product_eff['ROI'].values

In [24]:
# 4.5 训练，用什么模型更好？怎么调？目前用的是 RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y, 
    test_size=0.2,
    random_state=202504
)
model_cluster = RandomForestRegressor(
    n_estimators=100,
    random_state=202504,
    n_jobs=-1
)
model_cluster.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=202504)

In [25]:
# 4.6 拟合评估
def evaluate_regression_model(model, X_test, y_test, prefix='模型'):
    """
    回归模型评估函数，计算并打印 RMSE、R² 与 Adjusted R²。

    Para：
    -----------
    - model: 已训练的回归模型，需具有 predict 方法。
    - X_test: 测试集特征矩阵，shape (n_samples, n_features)。
    - y_test: 测试集真实值数组，shape (n_samples,)。
    - prefix: str，打印输出的前缀，用于区分不同模型。

    Return：
    -----------
    - tuple: (rmse, r2, adjusted_r2)
    """
    y_pred = model.predict(X_test)
    
    # RMSE 和 R²
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2   = r2_score(y_test, y_pred)
    # Adjusted R²
    n_samples, n_features = X_test.shape
    adjusted_r2 = 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)
    
    print(f"{prefix} Test RMSE: {rmse:.4f}")
    print(f"{prefix} Test R²:   {r2:.4f}")
    print(f"{prefix} Adjusted R²: {adjusted_r2:.4f}")
    
    return rmse, r2, adjusted_r2

rmse, r2, adj_r2 = evaluate_regression_model(
    model_cluster, X_test, y_test,
    prefix='簇-簇模型'
)

簇-簇模型 Test RMSE: 0.3008
簇-簇模型 Test R²:   0.9965
簇-簇模型 Adjusted R²: 1.0022


**有个问题：**就是模型到底怎么样才算好？rmse越低，r2越高就真的对我们的预测越好吗？

## -- 5. 商品介绍—达人匹配 --

In [44]:
def recommend_online(
    product_desc: str,
    product_km,                # 已训练的商品聚类器
    bert_tokenizer,             # 已加载的 BERT tokenizer
    bert_model,                 # 已加载的 BERT model
    scaler_prod,                # 商品侧数值特征 StandardScaler
    ohe_region: OneHotEncoder,  # 商品侧国家/地区 OneHotEncoder
    enc_p: OneHotEncoder,       # 商品簇 OneHotEncoder
    enc_c: OneHotEncoder,       # 达人簇 OneHotEncoder
    scaler_cre,                 # 达人侧数值特征 StandardScaler
    num_cre_cols,               # 达人数值特征列名列表
    X_creator_full,             # 全量达人特征稀疏矩阵
    creator_df: pd.DataFrame,   # 全量达人 DataFrame，含 'CreatorClusterID'
    M_ROI: pd.DataFrame,        # 簇交互矩阵 (Kp×Kc)
    model_cluster,              # 簇间回归模型
    top_k: int = 10,
    batch_size: int = 1024,
    max_length: int = 128
) -> pd.DataFrame:
    """
    在线推荐函数：输入新商品描述，按预测 ROI 从高到低输出 Top-K 达人列表。
    """
    # —— 1. 新商品特征构造 —— 
    # 1.1 文本向量
    inputs = bert_tokenizer(
        [product_desc],
        padding=True,
        truncation=True,
        return_tensors='pt',
        max_length=max_length
    )
    with torch.no_grad():
        outputs = bert_model(**{k: v.cpu() for k, v in inputs.items()})
    title_vec = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # (1, H)
    
    # 1.2 数值特征占位（训练时7维），这里用 0 填充
    num_prod_vals = np.zeros(len(scaler_prod.mean_))
    num_prod_scaled = scaler_prod.transform([num_prod_vals])       # (1, 7)
    
    # 1.3 时间特征占位（训练时 2 维 month & weekday），用 0 填充
    time_vals = np.array([[0, 0]])                                 # (1, 2)
    
    # 1.4 地域 one-hot（训练时 R 维），用空字符串填充
    region_ohe = ohe_region.transform(pd.DataFrame([['']], columns=['国家/地区']))
    
    # 1.5 合并所有商品特征（顺序必须与训练时一致）
    X_new_product = sparse.hstack([
        sparse.csr_matrix(title_vec),        # H
        sparse.csr_matrix(num_prod_scaled),  # 7
        sparse.csr_matrix(time_vals),        # 2
        sparse.csr_matrix(region_ohe)        # R
    ])
    assert X_new_product.shape[1] == X_product.shape[1], (
        f"新商品特征列数 {X_new_product.shape[1]} != 训练列数 {X_product.shape[1]}"
    )
    
    # 1.6 预测商品簇
    p_cluster = int(product_km.predict(X_new_product)[0])
    
    # —— 2. 粗粒度簇分数 —— 
    creator_clusters = creator_df['CreatorClusterID'].values
    coarse_scores = M_ROI.iloc[p_cluster, creator_clusters].values
    
    # —— 3. 细粒度回归打分 —— 
    N = X_creator_full.shape[0]
    preds = np.zeros(N, dtype=float)
    
    # 重复商品特征（用于拼接）
    # 包括簇 one-hot、文本/数值/时间/地域四部分
    P_ohe_new = enc_p.transform(pd.DataFrame([[p_cluster]], columns=['ProductClusterID']))
    prod_feat_full = sparse.hstack([
        sparse.csr_matrix(title_vec.repeat(batch_size, axis=0)),
        sparse.csr_matrix(num_prod_scaled.repeat(batch_size, axis=0)),
        sparse.csr_matrix(time_vals.repeat(batch_size, axis=0)),
        sparse.csr_matrix(region_ohe.repeat(batch_size, axis=0))
    ]).toarray()  # (batch_size, Dp)
    
    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        size = end - start
        
        # 3.1 簇 one-hot
        P_batch = np.repeat(P_ohe_new, size, axis=0)
        C_batch = enc_c.transform(
            pd.DataFrame(
                creator_clusters[start:end], 
                columns=['CreatorClusterID']
            ))
        
        # 3.2 原始商品特征
        Xp = prod_feat_full[:size, :]                             # (size, Dp)
        
        # 3.3 达人数值特征
        Xc = scaler_cre.transform(
            creator_df.iloc[start:end][num_cre_cols]
            .fillna(0).values
        )                                                         # (size, Dc)
        
        # 3.4 拼接 & 预测
        X_batch = np.hstack([P_batch, C_batch, Xp, Xc])
        preds[start:end] = model_cluster.predict(X_batch)
    
    # 最终分数由细粒度模型输出，可与 coarse_scores 融合
    final_scores = preds  # 或 0.5*coarse_scores + 0.5*preds
    
    # —— 4. 排序 & 输出 Top-K —— 
    top_idx = np.argsort(final_scores)[-top_k:][::-1]
    recommendations = creator_df.iloc[top_idx].copy()
    recommendations['predicted_ROI'] = final_scores[top_idx]
    
    return recommendations.reset_index(drop=True)

In [45]:
recs = recommend_online(
    product_desc="A new eletronic fan",
    product_km=product_km,
    bert_tokenizer=tokenizer,
    bert_model=bertmodel,
    scaler_prod=scaler_prod,
    ohe_region=ohe_region,
    scaler_cre=scaler_cre,
    num_cre_cols = num_cre_cols,
    enc_p=enc_p,
    enc_c=enc_c,
    X_creator_full=X_creator,
    creator_df=creator_df,
    M_ROI=M_ROI,
    model_cluster=model_cluster,
    top_k=10
)

In [46]:
print(recs[['Creator username','predicted_ROI']])

  Creator username  predicted_ROI
0      recosnfinds      19.882860
1      czarin0aa_c      19.795395
2       ellenia0aa      19.739544
3        shiimuran      19.739544
4    recosbysheila      19.739544
5      courtnylove      19.739544
6       drew_shopp      19.739544
7         nzrmymnj      19.727060
8          eeeriel      19.652079
9      maryshop219      19.455083


**存在问题：** 因为训练的时候部分达人的ROI很高，其他维度信息却很稀疏。最终输出老是会重复那几个ROI高的达人